In [1]:
# import sys
# !{sys.executable} -m pip install python-dotenv
# !{sys.executable} -m pip install google-api-python-client
# !{sys.executable} -m pip install iteration-utilities

In [2]:
# Code Link - https://github.com/onlyphantom/youtube_api_python/blob/main/yt_public.py

In [3]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
from iteration_utilities import unique_everseen
import csv
from tqdm import tqdm

In [4]:
comments = []

In [5]:
def process_comments(response_items, csv_output=False):

    for res in response_items:

        # loop through the replies
        if 'replies' in res.keys():
            for reply in res['replies']['comments']:
                comment = reply['snippet']
                comment['commentId'] = reply['id']
                comments.append(comment)
        else:
            comment = {}
            comment['snippet'] = res['snippet']['topLevelComment']['snippet']
            comment['snippet']['parentId'] = None
            comment['snippet']['commentId'] = res['snippet']['topLevelComment']['id']

            comments.append(comment['snippet'])

    if csv_output:
         make_csv(comments)
    
#     print(f'Finished processing {len(comments)} comments.')
    return comments

In [6]:
def make_csv(comments, channelID=None):
    header = comments[0].keys()

    if channelID:
#         filename = f'comments_{channelID}_{today}.csv'
        filename = f'comments.csv'
    else:
#         filename = f'comments_{today}.csv'
        filename = f'comments.csv'

    with open(filename, 'a', encoding='utf8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=header, extrasaction='ignore')
        writer.writeheader()
        writer.writerows(comments)

In [7]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

In [8]:
#create a service object for interacting with the YouTube Data API
youtube = build("youtube", "v3", developerKey=API_KEY, cache_discovery=False)

In [9]:
def comment_threads(videoID, to_csv=False):
    
    comments_list = []
    
    request = youtube.commentThreads().list(
        part='id,replies,snippet',
        videoId=videoID,
    )
    response = request.execute()
    comments_list.extend(process_comments(response['items']))

    # if there is nextPageToken, then keep calling the API
    while response.get('nextPageToken', None):
        request = youtube.commentThreads().list(
            part='id,replies,snippet',
            videoId=videoID,
            pageToken=response['nextPageToken']
        )
        response = request.execute()
        comments_list.extend(process_comments(response['items']))

    comments_list = list(unique_everseen(comments_list))

    print(f"Finished fetching comments for {videoID}. {len(comments_list)} comments found.")
    
    if to_csv:
#         make_csv(comments_list, videoID)
        make_csv(comments_list)
    return comments_list

In [10]:
video_ids = [
    "OUqqWRfCE0w", "CmOqJCsFEMo", "xncAlCvIqeI", "JrPhT1ggbR4",
    "JDz9SLo10pM", "BldgfZ1nfIY", "0rBn-_aVMuY", "GnUjvTZWGDQ", "BNZyfikdO9I",
    "pE3A75jHWsw", "P1ttJgHR0uY", "jNYyApxcpOU", "L8SuDRlpKBM", "jjQiurszvLs",
    "I_N04KkoYPk", "tO--TeUZ0Ik", "vFLbvgoJqzo", "6pTdreSJmtE", 
    "HsGa76se5Ns", "saJ7uiVwtEI", "QsW-ygDmhK8", "L_AVwEDNr70", "XTVOPH8AHEM",
    "LJ2U78nOi7c", "ZmblPQ1XR7k", "A6-yGHgrst4", "KfyYF5_pDzY", "xKSwVOzsOFk",
    "EglsC8Xix8s", "s8vtfwT2GW8", "txR94UXx-5U", "AxGRuCTxmkM"
]


In [ ]:
%%time
if __name__ == '__main__':
    # get comments
#     response = comment_threads(videoID='7Kt6ouYqacQ', to_csv=True)
#     print(response)
    for video_id in tqdm(video_ids, desc='Processing videos'):
        response = comment_threads(videoID=video_id, to_csv=True)

Processing videos:   2%|▉                                                             | 1/64 [01:03<1:07:11, 63.99s/it]

Finished fetching comments for rlKvtJlQc8U. 3095 comments found.


Processing videos:   3%|██                                                              | 2/64 [01:16<34:32, 33.43s/it]

Finished fetching comments for WGiEH-xrW74. 3392 comments found.


Processing videos:   5%|███                                                             | 3/64 [01:19<20:12, 19.87s/it]

...

Processing videos:  45%|████████████████████████████▌                                  | 29/64 [14:55<33:07, 56.79s/it]

Finished fetching comments for y9PgmL3cYiU. 9680 comments found.


Processing videos:  48%|██████████████████████████████▌                                | 31/64 [15:44<21:15, 38.66s/it]

Finished fetching comments for 09CgkWyqRO8. 9711 comments found.
Finished fetching comments for wuaAbcN13KE. 11394 comments found.


Processing videos:  50%|██████████████████████████████                              | 32/64 [22:51<1:22:51, 155.36s/it]

In [ ]:
'''
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
from iteration_utilities import unique_everseen
import csv
comments = []
def make_csv(comments, channelID=None):
    header = comments[0].keys()

    if channelID:
#         filename = f'comments_{channelID}_{today}.csv'
        filename = f'comments.csv'
    else:
#         filename = f'comments_{today}.csv'
        filename = f'comments.csv'

    with open(filename, 'w', encoding='utf8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=header, extrasaction='ignore')
        writer.writeheader()
        writer.writerows(comments)
def comment_threads(videoID, to_csv=False):
    
    comments_list = []
    
    request = youtube.commentThreads().list(
        part='id,replies,snippet',
        videoId=videoID,
    )
    response = request.execute()
    comments_list.extend(process_comments(response['items']))

    # if there is nextPageToken, then keep calling the API
    while response.get('nextPageToken', None):
        request = youtube.commentThreads().list(
            part='id,replies,snippet',
            videoId=videoID,
            pageToken=response['nextPageToken']
        )
        response = request.execute()
        comments_list.extend(process_comments(response['items']))

    comments_list = list(unique_everseen(comments_list))

    print(f"Finished fetching comments for {videoID}. {len(comments_list)} comments found.")
    
    if to_csv:
#         make_csv(comments_list, videoID)
        make_csv(comments_list)
    return comments_list
if __name__ == '__main__':
    # get comments
    response = comment_threads(videoID='G4EVRUFttG0', to_csv=True)
    print(response)

'''